### 데이터세트 결합 : Concat과 Append
판다스는 이러한 유형의 데이터 랭글링을 빠르고 간단하게 할 수 있는 함수와 메서드를 제공한다.  
*랭글링: 원데이터(raw data)를 또 다른 형태로 전환하거나 매핑하는 과정을 말하는 것으로 또 다른 말로 data munging 데이터 먼징이라고 함 

### 복습 NumPy 배열 연결

In [52]:
import pandas as pd
import numpy as np

x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

첫 번째 인수는 연결한 배열의 리스트나 튜플이다 axis 키워드를 사용해서 결과를 어느 축에 따라 연결할 것인지 결정 할  수 있다.

In [53]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

In [54]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [55]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

In [56]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

### pd.concat을 이용한 간단한 연결
NumPy와 구문이 비슷하지만 다양한 옵션을 가진 pd.concat()함수가 있다.

In [57]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [58]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


### 인덱서 복제
np.concatenate와 pd.concat의 중요한 차이는 Pandas에서의 연결은 그 결과가 복제된 인덱스를 가지더라도 인덱스를 유지하는데 있다.

In [59]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


결과에서 인덱스가 반복된다. 이것은 데이터프레임내에서는 유효하지만 결과가 바람직하지 않은 경우가 종종 있다. pd.concat()은 이 문제를 처리하는 몇가지 방법을 제공한다.  
1. 반복을 에러로 잡아낸다.  
pd.concat()의 결과에서 인덱스가 겹치지 않는지 간단히 검증하고 싶으면 verify_integrity 플래그를 지정하면된다. 이 플래그를 True로 설정하면 연결 작업에서 중복 인덱스가 있을 때 예외가 발생한다. 확인을 위해 오류를 잡아내고 메세지를 출력하자

In [60]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


2. 인덱스를 무시한다.  
인덱스 자체가 중요하지 않은 겨웅에는 그냥 인덱스를 무시하고 싶을 것이다. ignore_index플래그를 사용해 이 옵션을 지정할 수 있다. 이 플래그를 True로 설정하면 연결작업은 결과 Series에 새로운 정수 인덱스를 생성한다.

In [61]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


3. 다중 인덱스 키를 추가한다.  
데이터 소스에 대한 레이블을 지정하는데 keys옵션을 사용하는 것이다. 결과는 데이터를 포함하는 계층적 인덱스를 가진 시리즈가 된다.

In [62]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

결과는 다중 인덱스를 가지는 DataFrame이며 147쪽 계층적 인덱싱에서 살펴본 도구를 사용해 이 데이터를 관심 있는 표현 방식으로 전환할 수 있다.

### 조인을 이용한 연결

In [63]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

In [64]:
print(df5); print(df6), print(pd.concat([df5, df6]));

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


채울 값이 없는 항목은 기본적으로 NA값으로 채워진다. 이 값을 바꾸려면 연결 함수의 join과 join_axes매개변수에 대한 여러 옵션 중 하나를 지정하면 된다. 기본적으로 조인은 입력 열의 합집합(join='outer')이지만 join-'inner'를 사용해 이를 열의 교집합으로 변경할 수 있다

In [65]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

다른 방식은 인덱스 객체 목록을 취하는 join_axes 인수를 사용해 남은 열의 인ㄷ게스를 직접 지정하는 것이다. 예제에서는 반환된 열이 첫번째 입력값의 열과 동일해야 한다고 지정할 것이다. pd.concat함수의 옵션을 조합하면 두 데이터세트를 연결할 때 다양한 동작이 가능하다. 데이터 이러한 도구를 사용할때 이 점을 염두해 둔다.

### append()매서드
배열을 직접 연결하는 것이 매우 일반적이라서 Series와 DataFrame객체는더 적은 키 입력으로 똑같은 작업을 수행할 수 있는 append매서드를 가지고 있다. 예를 들어 pd.concat([df1,df2])를 호출하지 않고 간단하게 df1.append(df2)를 호출할 수 있다.

In [66]:
print(df1);print(df2);print(df1.append(df2))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


파이썬 리스트의 append(), extend()매서드와 달리 판다스의 append()매서드는 원래의 객체를 변경하지 않는 대신 결합된 데이터를 가지는 새로운 객체를 만든다. 이 방법 역시 새 인덱스와 데이터 버퍼를 생성하기 때문에 매우 효율적인 방식이라고 보기는 어렵다. 따라서 append 연산을 여러 번 수행할 계획아라면 일반적으로 DataFrame의 목록을 만들고 그것들을 concat()함수에 한번 전달하는 것이 더 바람직 하다.

### 데이터세트 결합하기: 병합과 조인
Pandas가 제공하는 기본 기능의 하나는 고성능 메모리 조인과 병합 연산이다.

### 관계 대수
pd.merge()에는 관계 데이터(relational data)를 조작하는 규칙의 정형 집합이자 대부분의 데이터베이스에서 사용할 수 있는 연산의 개념적 기반을 형성하는 관계 대수(relational algebra)의 하위 집합에 해당하는 행위가 구현돼 있다. 관계 대수의 강점은 데이터셋에 대한 복잡한 연산의 기본 구성요소가 되는 몇 가지 기초 연산을 제안한다는 것이다. 데이터베이스나 다른 프로그램에서 효율적으로 구현된 기초 연산의 어휘를 사용하면 매우 복잡한 작업을 다양하게 수행할 수 있다

### Join 작업의 분류
pd.merge() 함수는 일대일, 다대일, 다대다 조인 같은 여러 조인유형을 구현한다. 이 세가지 유형 모두 pd.merge() 인터페이스에서 호출을 할 수 있다.

### 일대일 조인
Concat, Append에서본 열 단위의 연결과 여러 면에서 매우 유사한 일대일 조인이다.

In [67]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [68]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [69]:
df3 = pd.merge(df1,df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### 다대일 Many to one 조인
다대일 조인은 두 개의 키 열 중 하나가 중복된 항복을 포함하는 경우의 조인을 의미한다. 다대일 조인의 경우 결과 DataFrame은 이 중복 항목을 타당한 것으로 보존한다. 다대일 조인에 대한 다음 예제를 보자

In [70]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

### 다대다 Many to many 조인
왼쪽 배열의 키열에 모두 중복 항목이 존재하면 결과는 다대다 병합이 된다. 구체적인 예제를 보자

In [71]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


이 세가지 유형의 조인은 다른 Pandas 도구와 함께 사용해 다양한 기능을 구현할 수 있다. 하지만 실제 데이터 세트가 여기서 다룬 것 만큼 깨끗한 것은 드물다. 조인 연산이 동작하는 방식을 조정할 수 있게 해주는 pd.merge()가 제공하는 몇 가지 옵션을보자

### 병합 키 지정
앞에서 pd.merge()의 기본 동작 방식을 알아보았다. pd.merge()는 두 개의 입력값 사이에 일치하는 하나 이상의 열 이름을 찾아 그것을 키로 사용한다. 그러나 열 이름이 그렇게 잘 일치하는 경우는 흔하지 않으며 pd.merge()가 이 문제를 처리하기 위한 다양한 옵션을 제공한다.

### on 키워드
가장 간단한 방법은 열 이름이나 열 이름의 리스트를 취하는 on 키워드를 사용해 키 열의 이름을 명시적으로 지정하는 것이다.

In [72]:
print(df1); print(df2); print(pd.merge(df1, df2, on='employee'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


### left_on과 right_on키워드
때로는 다른 열 이름을 가진 두 데이터 세트를 병합하고 싶을 수도 있다. 예를 들어 직원 이름 레이블이 employee가 아니라 name인 데이터세트를 가지고 있는 경우가 그렇다. 이 경우 left_on과 right_on키워드를 사용해 두 열 이름을 지정할 수 있다.

In [73]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

그 결과는 불필요하게 중복된 열을 갖게되며, 필요한 경우 데이터프레임의 drop() 매서들르 사용해서 삭제할 수 있다.

In [74]:
pd.merge(df1,df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### left_index와 right_index 키워드
때로는 열을 병합하는 대신 인덱스로 병합해야 하는 경우도 있다. 예를 들어

In [75]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a);print(df2a)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


여기서 pd.merge()의 left_index나 right_index를 지정해 병합 키로 인덱스를 사용할 수 있다.

In [76]:
print(df1a);print(df2a);
print(pd.merge(df1a,df2a,left_index=True, right_index=True))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


편의를 위해 데이터프레임은 기본적으로 인덱스 기반으로 조인하는 병합을 수행하는 join() 매서드를 구현한다.

In [77]:
print(df1a);print(df2a);print(df1a.join(df2a))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


인덱스와 열을 섞고자 한다면 left_index를 right_on과 결합하거나 left_on을 right_index와 결합해 원하는 결과를 얻을 수 있다

In [78]:
pd.merge(df1a,df3,left_index=True, right_on='name')

,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


이 옵션 모두 다중 인덱스나 다중 열에서도 동작한다. 이 해우이에 대한 인터페이스는 매우 직관적이다. 더 자세한 내용은 Pandas문서의 '병합. '조인 연결'을 참고

### 조인을 위한 집합 연산 지정하기

In [79]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

여기서는 'name'에서 공통 항목으로 유일하게 Mary를 가지고 있는 두 데이터 세트를 병합했다. 기본적으로 결과에는 입력값의 두 집합에 대한 교집합이 들어가는데 이것이 바로 내부 조인(inner join)이다. 기본적으로 'inner'로 설정된 how 키워드를 사용해 명시적으로 지정할 수 있다.

In [80]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


how 키워드의 다른 옵션으로 'outer', 'left', 'right'가 있다. 외부 조인(outer join)은 입력 데이터 열의 합집합으로 조인한 결과를 반환하고 누락된 값은 NA로 채운다.

In [81]:
pd.merge(df6,df7, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


왼쪽 조인(left join)과 오른쪽 조인(right join)은 각각 왼쪽 항목과 오른쪽 항목을 기준으로 조인한다.

In [82]:
pd.merge(df6,df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


### 열 이름이 겹치는 경우: suffixes키워드
마지막으로 두개의 입력 DataFrame이 충돌하는 열 이름을 가진 경우를 살펴보자

In [83]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

결과값에 동일한 rank라는 이름이 있기 때문에 결과열을 고유하게 만들기 위해서 _x, _y로 구분하는 것을 볼 수 있다. 이를 suffixes 키워드를 이용해 접미사를 별도로 지정할 수 있다.

In [84]:
pd.merge(df8,df9, on="name", suffixes=["_L","_R"])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


### 예제: 미국 주 데이터
병합과 조인은 서로 다른 소스에서 나온 데이터를 연결할 때 가장 자주 사용한다. 여기서는 미국 주와 주별 인구 데이터를 이용한 예제를 보자. 

In [89]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

먼저 인구에 전체 주의 이름을 제공하는 다대일 병합으로 시작한다 여기서는 pop의 state/regin열과 addrevs의 abbreviation열을 기준으로 병합하려고 한다. 레이블이 일치하지 않는다는 이유로 데이터가 제거되지 않도록 how=outer을 사용한다.

In [90]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
#merged = merged.drop('abbreviation',1) # 중복열 제거
merged = merged.drop(columns='abbreviation') #<-최신버전은 이렇게 씀
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [91]:
#코드에 불일치하는 항목이 있는지 다시 한번 보자 널 값을 가진 행을 찾으면 된다.
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

일부 population 정보가 널 값이다. 어떤 항목이 널이지 확인해본다

In [92]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


인구 데이터에서 널 값은 모두 2000년 이전의 푸에르토리코에서 비롯되었다. 이는 원본 소스에 사용할 수 없는 데이터가 없기 때문인 듯 하다. 더 중요한 것은 신규 state항목도 널 값인 거을 볼 수 있으며, 이는 abbrevs 키에 해당 항목이 없음을 의미한다. 이 일치되는 값이 없는 지역이 어디인지 보자

In [94]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

이렇게 되는 이유는 예제에 사용한 인구 데이터가 푸에르토리코(PR)와 전체 미국에 대한 항목을 포함하고 있지만 이 항목들이 주 이름의 약어 키(abbreviation key)에는 등장하지 않는다. 이 문제는 적절한 항목을 채워 넣음으로 해결 할 수 있다.

In [95]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

state열에 더이상 널값이 없다. 이제 비슷한 절차를 이용해 면적 데이터가 포함된 결과를 병합할 수 있다. 결과값을 검사해 보면 양쪽의 state 열을 기준으로 조인하고 싶을 것이다.

In [96]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


다시 말하지만, 일치하지 않는 항목이 있는지 보기 위해 널 값 여부를 확인하자.

In [97]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

area값에 널값이 있다. 어느 지역이 누락되었는지 보자

In [99]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

area DataFrame이 전체 미국 면적을 담고 있지 않음을 알 수 있다. 적절한 값(예를 들어, 모든 주 면적의 합계 사용)을 삽입할 수도 있지만, 이 경우에는 미국 전체 인구 밀도가 현재 논의하는 내용과 관련이 없기 때문에 그냥 널 값을 지워본다

In [100]:
final.dropna(inplace=True)
final

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2491,PR,under18,2010,896945.0,Puerto Rico,3515.0
2492,PR,under18,2011,869327.0,Puerto Rico,3515.0
2493,PR,total,2011,3686580.0,Puerto Rico,3515.0
2494,PR,under18,2012,841740.0,Puerto Rico,3515.0


이제 필요한 데이터를 갖췄다. 궁금한 질문에 답변하기 위해 먼저 2010년과 전체인구(total population)에 해당하는 데이터 부분을 선택하자. 이 작업을 신속히 처리하기 위해 여기서는 query()함수를 사용하겠다. (이 함수를 사용하려면 numexpr 패키지가 설치되어 있어야 한다. eval()과 query()참고

In [102]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


인구 밀도를 계산하고 그것을 순서